## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-22-19-01-32 +0000,nypost,"Fed’s preferred inflation gauge ticks up, dent...",https://nypost.com/2026/01/22/business/feds-pr...
1,2026-01-22-18-57-41 +0000,nyt,What to Know About the Elizabeth Smart Kidnapp...,https://www.nytimes.com/2026/01/22/us/elizabet...
2,2026-01-22-18-57-33 +0000,nyt,Live Updates: Zelensky Criticizes European All...,https://www.nytimes.com/live/2026/01/22/us/tru...
3,2026-01-22-18-57-11 +0000,eltecolote,Viral ‘Carpool’ photographs at SFMOMA capture ...,https://eltecolote.org/content/en/carpool-alej...
4,2026-01-22-18-57-00 +0000,wsj,U.S. Weighs Complete Military Withdrawal From ...,https://www.wsj.com/world/middle-east/u-s-weig...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2420/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
25,trump,94
109,greenland,33
47,davos,28
74,ice,19
5,up,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
248,2026-01-22-07-39-58 +0000,bbc,Trump says 'framework of a future deal' discus...,https://www.bbc.com/news/articles/cgezx40r7d7o...,193
190,2026-01-22-11-19-40 +0000,nypost,"Trump claims ‘Iran does want to talk, and we’l...",https://nypost.com/2026/01/22/us-news/trump-cl...,181
142,2026-01-22-14-32-36 +0000,nypost,Trump vows US will have ‘total access’ to Gree...,https://nypost.com/2026/01/22/us-news/trump-vo...,178
399,2026-01-21-19-14-45 +0000,nyt,Did the U.S. Ever Own Greenland? Fact-Checking...,https://www.nytimes.com/2026/01/21/us/politics...,173
215,2026-01-22-10-34-23 +0000,nypost,Trump unveils ‘Board of Peace’ in Davos ceremo...,https://nypost.com/2026/01/22/us-news/trump-un...,170


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
248,193,2026-01-22-07-39-58 +0000,bbc,Trump says 'framework of a future deal' discus...,https://www.bbc.com/news/articles/cgezx40r7d7o...
215,76,2026-01-22-10-34-23 +0000,nypost,Trump unveils ‘Board of Peace’ in Davos ceremo...,https://nypost.com/2026/01/22/us-news/trump-un...
281,68,2026-01-22-02-17-39 +0000,cbc,Former Uvalde school police officer found not ...,https://www.cbc.ca/news/world/uvalde-police-sc...
46,60,2026-01-22-17-47-41 +0000,startribune,"Live: Activists arrested after church protest,...",https://www.startribune.com/ice-raids-minnesot...
273,54,2026-01-22-03-11-00 +0000,wsj,The Qatari-donated plane that is being overhau...,https://www.wsj.com/politics/national-security...
112,52,2026-01-22-15-43-47 +0000,nypost,Barron Trump told cops his body was ‘racing wi...,https://nypost.com/2026/01/22/us-news/barron-t...
190,42,2026-01-22-11-19-40 +0000,nypost,"Trump claims ‘Iran does want to talk, and we’l...",https://nypost.com/2026/01/22/us-news/trump-cl...
113,39,2026-01-22-15-43-09 +0000,nyt,Trump’s Rift With Europe Is Clear. Europe Must...,https://www.nytimes.com/2026/01/21/world/europ...
34,39,2026-01-22-18-04-49 +0000,nypost,Wild video shows Long Island cops respond to b...,https://nypost.com/2026/01/22/us-news/long-isl...
98,36,2026-01-22-16-18-59 +0000,nypost,2 dead in apparent botched NYC murder-suicide ...,https://nypost.com/2026/01/22/us-news/2-dead-i...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
